In [ ]:
import os
from datetime import datetime
import numpy as np 
import gmat_nav as gnav
# Directory location of this Jupyter notebook tutorial
file_dir = os.path.abspath('') #os.path.dirname(os.path.abspath(__file__))
# --------------- USER DEFINED PATHS (USER -- PLEASE LOOK HERE!) ------------------# 
gmat_root_dir = '/home/natsubuntu/Desktop/SysControl/estimation/CauchyCPU/CauchyEst_Nat/GMAT'
gmat_data_dir = file_dir + "/data"
eop_filepath = gmat_data_dir + "/eop_file.txt"
spaceweather_filepath = gmat_data_dir + "/SpaceWeather-v1.2.txt"
gnav.check_then_load_gmatpy(gmat_root_dir, eop_filepath, spaceweather_filepath)
gmat = gnav.gmat # our handle to the gmat module, loaded through gnav
gmat.Clear() # clears any junk lying around 

Welcome to tutorial 2! In this tutorial, we will use the GMAT API to simulate an earth orbiting satellite. We will see how to propagate the satellite around earth, model an atmosphere, solar radiation pressure, and other forces that interact with our orbiter.

A quick note: the author of these tutorials has attempted to explain many of the high level GMAT (Python API) concepts. However, detail and explanation of some parameters not deemed "high-level" have been omitted. One can find an extensive reference manual by opening up the GMAT GUI, clicking "Reference Guide" under "Links and Resources", and searching for the topic of interest. It is truly an extensive documentation source on all concepts GMAT, but is little help for understanding how the Python API works. Thats where these tutorials comes in.

Lets get started. First on the docket: as we are constructing an earth-orbiting satellite, we need to specifically tell GMAT what EOP file to use. We also need to specify an ephemeris source. This can be done by configuring your "solar system" through the GMAT moderator as

In [ ]:
mod = gmat.Moderator.Instance() # Instance of the GMAT Moderator
ss = mod.GetDefaultSolarSystem() # Get solar system object
ss.SetField("EphemerisSource", "DE421") # Tell GMAT the ephemeris source, this is a standard choice, and included with GMAT
earth = ss.GetBody('Earth') # Get reference to the Earth 
earth.SetField('EopFileName', eop_filepath) # Set EOP params for an Earth Orbiter

The above is a crucial (but under-reported) step when setting up an earth-orbiting satellite. Specifying the EOP file becomes unavoidable when processing GPS measurements (which are typically given in the Earth Body Fixed Frame and not an Earth Inertial Frame -- much more on this later). If not done, you may recieve a headache, courtesy of GMAT, when converting between coordinate frames. So, this point is belabored up front.

Now that our solar system is properly setup for earth orbit, lets go ahead and start defining a satellite that will orbit earth.

In [ ]:
# Create a satellite, using UTC Gregorian Time, with Cartesian State type, and in the EarthMJ2000Eq (Inertial) frame
sat = gmat.Construct("Spacecraft", "Fermi") # Naming the spacecraft 'Fermi'
sat.SetField("DateFormat", "UTCGregorian") # Setting DateFormat to UTCGregorian
sat.SetField("CoordinateSystem", "EarthMJ2000Eq") # Setting CoordinateSystem to EarthMJ2000Eq
sat.SetField("DisplayStateType","Cartesian") # Setting DisplayStateType to Cartesian

GMAT's Construct(...) function is used to return a GMAT Object. The first argument to Construct(...) is the GMAT Object you wish to construct. The second argument is a user-provided name. The Spacecraft object is what we need to define an earth-orbiting satellite.

Almost every GMAT object returned by Construct(...) will have a method called SetField(...), which allows you to set its various attributes. The first argument of SetField(...) is the field itself you wish to set, while the second argument is its value. The above is fairly standard, for an earth orbiter.

We need to specify the satellite properties now, by using its SetField(...). For a basic satellite, we will need to specify its Epoch (i.e., time), (dry)mass, nominal coefficient of drag, nominal coefficient of reflectivity, satellite area, mass, and its state (position vector, velocity vector). We can do so as 

In [ ]:
# Define the time (i.e, your epoch) and your state
t0 = "10 Jul 2023 19:34:54.000" # Initial Time in UTC Gregorian!
pos3 = np.array([4.9962452882705193e+03,3.8779464630861030e+03,2.7360432364171807e+03]) # approx a 550 (units -> kilometer) orbit
vel3 = np.array([-5.0280935744461930e+00,5.5759213419992673e+00,1.2698611722905329e+00]) # speed (units -> kilometers/second) needed to 
x0 = np.concatenate((pos3, vel3)) # Your initial state vector
Cd0 = 2.1 # nominal coefficient of drag # unitless
Cr0 = 0.75 # nominal coefficient of reflectivity # unitless
A = 14.18 # (Drag) Area of the satellite # units -> meters
m = 3995.6 # (dry) mass of the satellite # units -> kilograms
sat_ID = '2525' # pick your favorite number, as a string

sat.SetField("Epoch", t0)
sat.SetField("X", x0[0])
sat.SetField("Y", x0[1])
sat.SetField("Z", x0[2])
sat.SetField("VX", x0[3])
sat.SetField("VY", x0[4])
sat.SetField("VZ", x0[5])
sat.SetField("Cd", Cd0)
sat.SetField("Cr", Cr0)
sat.SetField("DragArea", A)
sat.SetField("SRPArea", A) # Equal SRP (solar radiation pressure) area as DragArea
sat.SetField("DryMass", m)
sat.SetField("Id", sat_ID)

We may also wish to specify that the satellite has a fuel tank, which adds additional mass due to the weight of its propellant. We can put this into the works as 

In [ ]:
fuel_mass = 359.9 # units -> kilograms
fueltank = gmat.Construct("ChemicalTank", "FuelTank") # Naming our ChemicalTank 'FuelTank'
fueltank.SetField("FuelMass", fuel_mass)
sat.SetField("Tanks", "FuelTank") # Assign ChemicalTank Object to our Satellite Object by its name

Above, the last line of our code block assigns the Fuel Tank (ChemicalTank) Object to the Satellite Object. GMAT's backend can do so by using the names you've attributed to the objects themselves.

You may be wondering "well...what are the configurable parameters of each GMAT Object?"
> You can use an objects "obj.Help()" method to, well, help you out here. It will print out some commentary on what this object is, as well as its parameters.

You may also be wondering, "well...how do I know my parameters have actually been accepted by GMAT?"

The answer is twofold: 
> 1.) Usually, GMAT will throw a (possibly cryptic) error if you do something illegal. In the experience of the author of this tutorial, this isn't always the case: sometimes GMAT simply dismisses mis-specified parameters and does not notify you.

<br>

> 2.) Use the objects "obj.GetGeneratingString(0)" method, which lists the objects parameters, as well as values they are set to. This makes it very easy to check and see if things were set correctly when configuring your orbital environment.

For example, you can get nosey and view the .Help() and .GetGeneratingString(0) for the objects we have created so far by uncommenting any one of the lines below, and rerun the code block:

In [ ]:
# HELP STRINGS
# Printing out Solar Sys Help() String
#ss.Help()
# Printing out Earth's Help() String
#earth.Help()
# Printing out Earth's Help() String
#sat.Help()
# Printing out ChemicalTank's Help() String
#fueltank.Help()

# GetGeneratingString STRINGS
# Printing out Solar Sys GetGeneratingString(0) String
#ss.GetGeneratingString(0)
# Printing out Earth's GetGeneratingString(0) String
#earth.GetGeneratingString(0)
# Printing out Earth's GetGeneratingString(0) String
#sat.GetGeneratingString(0)
# Printing out ChemicalTank's GetGeneratingString(0) String
#fueltank.GetGeneratingString(0)

You can see that the Help() string output is more verbose than GetGeneratingString(0). Almost all of an object's parameters can be set using the "obj.SetField(FieldName, FieldValue)" method. Afterward, using "obj.GetGeneratingString(0)" allows you check this, pending SetField doesn't throw an error at you first.

Now we need to create a ForceModel Object. This tells GMAT explicitly what forces affect/perturb the satellite's motion in our simulated universe. Below is a fairly typical setup:

In [ ]:
# Create Force Model 
grav_model_degree_order = 70 # The larger, the higher fidelity
flux = 1370.052 # units -> Watts/m^2

# Force Model of the orbiter
fm = gmat.Construct("ForceModel", "TheForces")
fm.SetField("ErrorControl", "None")

# 1.) A 70x70 EGM96 Gravity Model
earthgrav = gmat.Construct("GravityField")
earthgrav.SetField("BodyName","Earth")
earthgrav.SetField("Degree", grav_model_degree_order) 
earthgrav.SetField("Order", grav_model_degree_order)
earthgrav.SetField("PotentialFile","EGM96.cof")
earthgrav.SetField("TideModel", "SolidAndPole")

# 2.) The Point Masses 
moongrav = gmat.Construct("PointMassForce")
moongrav.SetField("BodyName","Luna")
sungrav = gmat.Construct("PointMassForce")
sungrav.SetField("BodyName","Sun")

# 3.) Solar Radiation Pressure Model
srp = gmat.Construct("SolarRadiationPressure")
srp.SetField("SRPModel", "Spherical")
srp.SetField("Flux", flux)

# 4.) Atmospheric Drag Model
jrdrag = gmat.Construct("DragForce")
jrdrag.SetField("AtmosphereModel","JacchiaRoberts")
jrdrag.SetField("HistoricWeatherSource", 'CSSISpaceWeatherFile')
jrdrag.SetField("CSSISpaceWeatherFile", spaceweather_filepath)
atmos = gmat.Construct("JacchiaRoberts") #Atmospheric Density Model (ADM)
jrdrag.SetReference(atmos) # Assign ADM to Drag Model

# Add Force Objects to Force Model 
fm.AddForce(earthgrav)
fm.AddForce(moongrav)
fm.AddForce(sungrav)
fm.AddForce(jrdrag)
fm.AddForce(srp)
#fm.GetGeneratingString(0)

Lets quickly review what is going on in the above code block.

> 1.) Add Gravity Model for Central Body: Here, earth is our central body. This adds the force enacted on the satellite by Earth, as the satellite orbits it. The higher the degree/order of the model, the higher fidelity. From the experience of the author of these tutorials, 70 (degree/order) is a typical choice in practice.

<br>

> 2.) Add Point Masses: This adds the force(s) enacted on the satellite by (non-central) masses. This allows our simulation to be more realistic by constructing a (greater than) two body problem. Here, we are constructing a four body problem with the most dominant (non-central) forces the satellite is subject to (sun, moon). In total, Satellite + Earth + Sun + Moon yields a four body problem.

<br>

> 3.) Add Solar Pressure: Force caused by an exchange in momenta between the photons emitted by the Sun and the satellite’s surface. Note, you should pick a nominal value for the flux.

<br>

> 4.) Add Atmospheric Drag: Typically, this is the most dominant perturbation force acting on low Earth orbit (LEO) satellites. This force acts opposite to the direction of travel and is due to the satellite passing through (the thin) atmosphere at its orbital height. Accurate modelling of which is difficult. However, GMAT provides a high fidelity drag model known as Jacchia Roberts, but requires a space weather file. GMAT will use a default space weather source, if not provided. However, it is a best practice to specify it yourself. 

<br>

Now that our forces are setup, we need to construct a propagator. The propagator will take our force model and subject the satellite to these forces (as the satellite orbits) by means of a numerical integration scheme. The following sets this up:

In [ ]:
dt = 60.0 # units -> seconds (Our Time Step)
rk89_accuracy = 1.0e-13
max_attemps = 50

# Build Integrator
gator = gmat.Construct("RungeKutta89", "Gator")

# Build the propagation container that connect the integrator, force model, and spacecraft together
pdprop = gmat.Construct("Propagator","PDProp")  

# Create and assign a numerical integrator for use in the propagation
pdprop.SetReference(gator)
pdprop.SetField("InitialStepSize", dt)
pdprop.SetField("Accuracy", rk89_accuracy)
pdprop.SetField("MinStep", 0)
pdprop.SetField("MaxStep", dt)
pdprop.SetField("MaxStepAttempts", max_attemps)

# Assign the force model to the propagator
pdprop.SetReference(fm)

# The propagator also needs to know the object that is being propagated
pdprop.AddPropObject(sat)

# Setup the container which manages all objects being propagated, i.e., our satellite
psm = gmat.PropagationStateManager()
psm.SetObject(sat)
#psm.SetProperty("AMatrix") # will be needed later on
psm.BuildState()

# Assign PSM as the manager to our force model 
fm.SetPropStateManager(psm)
# Tell force model what the state of our propagated objects are
fm.SetState(psm.GetState())

The above is a somewhat messy sequence of object assignments, but if you squint at it long enough, it begins to make some sense. In play is our Satellite Object, Force Model Object, the Integrator Object, Propagator Object, and a Propagation State "Manager" Object. 


The Propagator needs to know what numerical integration scheme to use, so we assign the Integrator to it. The Runge Kutta integrator is typical, but there are others (such as Dormand–Prince). It also needs to know what object(s) it is propagating (here, our single satellite), and what forces are being modelled, i.e, the ForceModel. The Propagation State Manager keeps track of all these connections.

This completes our set-up. Now, a few commands are needed to tell GMAT you're ready to go.

In [ ]:
# Tell GMAT your're ready to go
gmat.Initialize()

##  Map the spacecraft state into the model
fm.BuildModelFromMap()
#  Load the physical parameters needed for the forces
fm.UpdateInitialData()

# Perform the integation subsysem initialization
pdprop.PrepareInternals()
# Refresh the integrator reference after initialization
gator = pdprop.GetPropagator()

Our "Final Product" is the 'gator' object above. We can use this object to propagate our satellite, and retrieve the updated satellite state after a propagation. The following shows how to propagate the satellite, and plot its state history:

In [ ]:
import matplotlib.pyplot as plt 
num_props = 200
states = [] 
times = [] 

# Begin Loop 

# Append initial state
states.append(np.array(gator.GetState())) # state is returned as a list 
times.append(gator.GetTime())
for i in range(num_props):
    # Step and Append 
    gator.Step(dt)
    states.append(np.array(gator.GetState()))
    times.append(gator.GetTime())

# Plot 3D Trajectory
states = np.array(states)
times = np.array(times)
fig = plt.figure() 
ax = fig.gca(projection='3d') 
ax.set_title('Satellite Trajectory')
ax.plot(states[:,0], states[:,1], states[:,2])
ax.set_xlabel('X (Km)')
ax.set_ylabel('Y (Km)')
ax.set_zlabel('Z (Km)')
# Plot Velocities
fig2 = plt.figure()
plt.suptitle("Velocities (X/Y/Z), respectively")
plt.plot(times, states[:,3], 'r') # X axis
plt.plot(times, states[:,4], 'g') # Y axis
plt.plot(times, states[:,5], 'b') # Z axis
plt.ylabel("Km/sec")
plt.xlabel("seconds")

# Used for comparison for next cell block
print("Initial State: ", x0)
print("Propagated State:", states[1])

Now that we've walked through setting this procedure up once, you can readily create a Satellite Object (for LEO satellites orbiting earth) through the gnav module, by using the class "gnav.EarthOrbitingSatellite(...)". We will use this clean (two-liner) call in the following tutorials. To conclude this tutorial, we'll see how to construct this object. It should look familiar to you now.

In [ ]:
# Scrap our previous gmat settings
gmat.Clear()

# Remember, your intial epoch time must be UTC Gregorian, otherwise use the time_convert(...) function in gnav to switch it!
t0 = datetime(year=2023, month=7, day=10, hour=19, minute=34, second=54, microsecond=0) #"10 Jul 2023 19:34:54.000" # Initial Time (string or datetime object) in UTC Greg

pos3 = np.array([4.9962452882705193e+03,3.8779464630861030e+03,2.7360432364171807e+03]) # approx a 550 (units -> kilometer) orbit
vel3 = np.array([-5.0280935744461930e+00,5.5759213419992673e+00,1.2698611722905329e+00]) # speed (units -> kilometers/second) needed to 
x0 = np.concatenate((pos3, vel3)) # Your initial state vector
Cd0 = 2.1 # nominal coefficient of drag # unitless
Cr0 = 0.75 # nominal coefficient of reflectivity # unitless
A = 14.18 # (Drag) Area of the satellite # units -> meters
m = 3995.6 # (dry) mass of the satellite # units -> kilograms
fuel_mass = 359.9 # units -> kilograms
solar_flux = 1370.052 # units -> Watts/m^2
dt = 60.0 # units -> seconds (Our Time Step)
earth_model_degree = 70 # (our grav_model_degree_order from before)
earth_model_order = 70 # (our grav_model_degree_order from before)
# Optional forces
with_jacchia = True 
with_SRP = True

# As this is a single satellite class object using GMAT, we do not truly need a 'spacecraft ID'
# Setting gmat_print=True will print the object Help() or GenereratingString() methods ...
#  ... giving you some extra confidence that parameters are set correctly
sat = gnav.EarthOrbitingSatellite(eop_filepath, spaceweather_filepath, gmat_print=False)
sat.create_model(t0, x0, Cd0, Cr0, A, m, fuel_mass, solar_flux, dt,
    earth_model_degree = earth_model_degree,
    earth_model_order = earth_model_order,
    with_jacchia = with_jacchia,
    with_SRP = with_SRP)
print("Initial State: ", x0)
print("Propagated State:", sat.step())

There are several more optional arguments you can pass to sat.create_model(...). These all correspond to integrator parameters. The default values, however, meet the mission specs of the Fermi Satellite.

We are now ready to introduce our estimation precursors! See Tutorial 3.